# **IMDB Recommender Chatbot Project**
## **IMDB Movie Recommendation System Using RAG**

### Project Overview
This project demonstrates my ability to build a custom recommendation system using Retrieval-Augmented Generation (RAG) with IMDB datasets. The system recommends movies based on user preferences or trending content.

The dataset includes:
- movies and tv shows
- produced in the US
- from 2014 on
- 100 additional titles from [The 100 Best Movies Of All Time](https://www.empireonline.com/movies/features/best-movies-2/)

### Objectives
1. Showcase proficiency in data cleaning, merging, and preprocessing.
2. Implement a RAG pipeline for movie recommendations.
3. Present results interactively and narratively.

## **PART 1 - Creating the Dataset**

This is a **standalone notebook** dedicated to the creation of the dataset that will be used to train the LLM model for movie recommendations.

Since collecting information using the OMDb API is a time-consuming process—estimated to take approximately **40 hours for around 30,000 movie IDs**—this notebook is intended to be run only once to generate and store the dataset.

Once the dataset is ready, all future work will be conducted using the preprocessed and saved dataset, eliminating the need to re-run this notebook. This ensures efficiency and prevents unnecessary API calls.

The final dataset will be saved in a structured format for easy retrieval and use in subsequent notebooks.

## **1. Data Preparation**

### **1.1 Import libraries**


In [1]:
import pandas as pd
import numpy as np
import logging
import os
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
import pickle

import openai

from sklearn.metrics.pairwise import cosine_similarity

# Set up logging to track progress and issues
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Import the OpenAI API key
from config_loader import load_config_value
openai_api_key = load_config_value("OPENAI_API_KEY")
omdb_key = load_config_value("OMDB_API_KEY")

In [3]:
# Local paths
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)

## **2. Data Collection**

### **2.1 Scraping Top 100 Iconic Movies from Empire Online**

The first step in our data collection process is to scrape the 100 most iconic movies from Empire Online. This ensures that we have a curated list of historically significant films to include in our recommendation system.

How It Works:
- The function scrape_empire_top_100(url) fetches the webpage and extracts movie titles and release years using BeautifulSoup and regex.
- The extracted movies are stored in a Pandas DataFrame for further processing.
- The data is then saved as a CSV file for easy reference.

In [5]:
def scrape_empire_top_100(url):
    """
    Scrape the 100 most iconic movies from the Empire Online website, extracting titles and release years.

    Args:
        url (str): URL of the Empire Online top movies page.

    Returns:
        pd.DataFrame: A DataFrame containing the scraped movie titles and years.
    """
    # Send a GET request to the URL
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        return None
    
    # Parse the webpage content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract movie titles and years, skipping the first <h2>
    movies = []
    h2_tags = soup.find_all('h2')[1:]  # Skip the first occurrence

    for item in h2_tags:
        title_text = item.get_text(strip=True)
        
        # Extract the title and year separately using regex
        match = re.search(r"\d+\)\s*(.+?)\s*\((\d{4})\)", title_text)
        if match:
            title, year = match.group(1), match.group(2)
            movies.append({'Title': title, 'Year': year})

    # Convert to a DataFrame
    movies_df = pd.DataFrame(movies)
    
    return movies_df

# URL of the Empire Online webpage
url = "https://www.empireonline.com/movies/features/best-movies-2/"

# Scrape the data
top_100_movies = scrape_empire_top_100(url)

# Save to a CSV file (optional)
if top_100_movies is not None:
    top_100_movies.to_csv("empire_top_100_movies.csv", index=False)
    print("Scraping completed and saved to 'empire_top_100_movies.csv'.")
else:
    print("Scraping failed.")

# Print first 10 rows
print(top_100_movies.head(10))

Scraping completed and saved to 'empire_top_100_movies.csv'.
                         Title  Year
0               Reservoir Dogs  1992
1                Groundhog Day  1993
2                 Paddington 2  2017
3                       Amélie  2001
4           Brokeback Mountain  2005
5                 Donnie Darko  2001
6  Scott Pilgrim Vs. The World  2010
7   Portrait Of A Lady On Fire  2020
8       Léon: The Professional  1994
9                        Logan  2017


### **2.2 Filtering and Merging IMDb Data**

Now that we have our top 100 movies, we need to fetch their corresponding IMDb metadata. To do this, we use IMDb’s title.basics.tsv dataset, which contains:
- Title Type (movie, tvSeries, etc.)
- Release Year
- Title Name
- Unique IMDb Identifier (tconst)

How It Works:
- Filters IMDb data to select:
- Movies (titleType == 'movie')
- Titles released from 2014 onward (to keep recent content relevant)
- Merges this dataset with the top 100 movies list, ensuring that only movies that exist in both sets are included.
- Extracts IMDb IDs (tconst) from:
- The filtered IMDb dataset
- The iconic movie dataset
- Combines the IDs, ensuring all relevant movies are included in our final list.

## **2.3 Fetching Additional Movie Data from OMDB API**

Now that we have identified relevant IMDb movie IDs, we use the OMDb API to fetch additional movie details, including:
- Genres, Directors, Actors
- Plot Summary
- IMDB Ratings and Votes
- Country of Production
- Awards and Poster URLs

How It Works:
- Loads the IMDb title.basics.tsv dataset and filters it.
- Extracts IMDb tconst IDs for:
- General filtered movies.
- The top 100 iconic movies.
- Calls the OMDb API for each tconst, retrieving structured movie metadata.
- Stores the data in a Pandas DataFrame for further use.

In [7]:
def collect_data_ids(basics_path, ratings_path, top_100_movies):
    """
    Filter IMDb datasets to collect movie IDs of interest.

    Args:
        basics_path (str): Path to the `title.basics.tsv` file.
        ratings_path (str): Path to the `title.ratings.tsv` file.
        top_100_movies (pd.DataFrame): DataFrame containing the top 100 iconic movies.
        
    Returns:
        List: A list containing the movie IDs.
    """
    
    
    ### 1. Load and Filter `title.basics.tsv`
    basics = pd.read_csv(basics_path, sep='\t', low_memory=False)
    basics_filtered = basics[
        (basics['titleType'] == 'movie') & 
        (basics['isAdult'] == '0') & 
        (basics['startYear'].apply(lambda x: x.isdigit() and int(x) >= 2014))
    ]

    ### 2. Load and Filter `title.ratings.tsv`
    ratings = pd.read_csv(ratings_path, sep='\t', low_memory=False)
    ratings_filtered = ratings[ratings['averageRating'] >= 7]

    ### 3. Join Basics and Ratings on `tconst`
    filtered_movies = basics_filtered.merge(ratings_filtered, on='tconst', how='inner')

    ### 4. Merge with Top 100 Iconic Movies
    top_100_movies['Year'] = top_100_movies['Year'].astype(str)  # Ensure year is a string
    iconic = basics.merge(top_100_movies, left_on=['originalTitle', 'startYear'], right_on=['Title', 'Year'], how='inner')
    iconic = iconic[iconic['titleType'] == 'movie']  # Keep only movies
    iconic = iconic.drop(columns=['Title'])  # Remove duplicate column

    ### 5. Combine IMDb and Iconic Movie IDs
    filtered_ids = filtered_movies['tconst'].tolist()
    iconic_ids = iconic['tconst'].tolist()
    movie_ids = list(set(filtered_ids + iconic_ids))  # Ensure unique IDs

    return movie_ids


In [9]:
basics_path = parent_path + '/IMDB_Dataset/title.basics.tsv'
ratings_path = parent_path + '/IMDB_Dataset/title.ratings.tsv'

movie_ids = collect_data_ids(basics_path, ratings_path, top_100_movies)

# Save the result to a .txt file
with open('movie_ids_list.txt', 'w+') as f:
    for id in movie_ids:
         f.write('%s\n' %str(id))

f.close()

In [10]:
def fetch_movie_data_from_omdb(movie_ids, api_key, save_path):
    """
    Fetch movie information from the OMDB API based on IMDb movie IDs.
    
    If a pickle file exists, it resumes from where it left off.

    Args:
        movie_ids (list): List of filtered movie IDs.
        api_key (str): Your OMDB API key.
        save_path (str): Path to save progress as a pickle file.

    Returns:
        pd.DataFrame: A DataFrame containing the fetched movie information.
    """
    
    # Try loading previously saved progress
    try:
        with open(save_path, "rb") as f:
            movie_data_list = pickle.load(f)
            fetched_ids = {movie["imdb_id"] for movie in movie_data_list}  # Track already fetched IDs
            print(f"Resuming from {len(movie_data_list)} movies...")
    except (FileNotFoundError, EOFError):
        movie_data_list = []
        fetched_ids = set()

    # Fetch only missing movie IDs
    remaining_ids = [m_id for m_id in movie_ids if m_id not in fetched_ids]

    for movie_id in tqdm(remaining_ids, desc="Fetching movie data"):
        print(f"Fetching data for movie ID {movie_id} from OMDb...")
        
        url = f"http://www.omdbapi.com/?apikey={api_key}&i={movie_id}&plot=full"
        response = requests.get(url)
        
        if response.status_code == 200:
            movie_data = response.json()
            
            if movie_data.get("Response", "False") == "True":
                # Store relevant fields
                movie_data_list.append({
                    "imdb_id": movie_id,
                    "title": movie_data.get("Title", None),
                    "year": movie_data.get("Year", None),
                    "genre": movie_data.get("Genre", None),
                    "director": movie_data.get("Director", None),
                    "actors": movie_data.get("Actors", None),
                    "plot": movie_data.get("Plot", None),
                    "country": movie_data.get("Country", None),
                    "awards": movie_data.get("Awards", None),
                    "poster": movie_data.get("Poster", None),
                    "rating": movie_data.get("imdbRating", None),
                    'metascore':movie_data.get("imdbRating", None),
                    "votes": movie_data.get("Metascore", None),
                    "type": movie_data.get("Type", None)
                })

                # Save progress every 10 movies
                if len(movie_data_list) % 100 == 0:
                    with open(save_path, "wb") as f:
                        pickle.dump(movie_data_list, f)
                    print(f"Progress saved after {len(movie_data_list)} movies.")
        else:
            print(f"Failed to fetch data for ID: {movie_id}, Status Code: {response.status_code}")

    # Final save after completion
    with open(save_path, "wb") as f:
        pickle.dump(movie_data_list, f)
    print("Final dataset saved!")

    # Convert to DataFrame and return
    movie_df = pd.DataFrame(movie_data_list)
    return movie_df

In [13]:
movie_ids.remove('tt4745832')
movie_ids.remove('tt4176748')

In [15]:
# Fetch movie data and generate a DataFrame
save_path = parent_path + "/movie_data.pkl"
movie_df = fetch_movie_data_from_omdb(movie_ids, omdb_key, save_path)

movie_df.to_csv(parent_path + "/filtered_movies.csv", index=False)

Resuming from 36400 movies...


Fetching movie data:   0%|                             | 0/2614 [00:00<?, ?it/s]

Fetching data for movie ID tt13701810 from OMDb...


Fetching movie data:   0%|                     | 1/2614 [00:00<20:05,  2.17it/s]

Fetching data for movie ID tt5080012 from OMDb...


...

Fetching data for movie ID tt5946852 from OMDb...


Fetching movie data: 100%|█████████████████▉| 2612/2614 [17:45<00:03,  1.94s/it]

Fetching data for movie ID tt21980046 from OMDb...


Fetching movie data: 100%|██████████████████| 2614/2614 [17:47<00:00,  2.45it/s]


Fetching data for movie ID tt17886916 from OMDb...
Final dataset saved!


In [23]:
# Load previously fetched movie data
with open(save_path, "rb") as f:
    movie_data_list = pickle.load(f)

# Convert to DataFrame
movie_df = pd.DataFrame(movie_data_list)
print(movie_df.head())

      imdb_id                      title  year               genre  \
0   tt4076360               Third Person  2015  Documentary, Drama   
1   tt3608918                  Azzurrina  2023              Horror   
2  tt15908496                Dear Jackie  2021         Documentary   
3  tt28378602  Zena s gumenim rukavicama  2023               Drama   
4   tt6340460        Los caminos de Cuba  2019         Documentary   

            director                                          actors  \
0       Sharon Luzon                                             N/A   
1  Giacomo Franciosa  Tatiana Luter, Paolo Stella, Gianfranco Terrin   
2        Henri Pardo  Ronald Jones, Ivan Livingstone, Majiza Philips   
3       Mario Sulina   Areta Curkovic, Sandra Loncaric, Miro Cabraja   
4      Luciano Nacci                                             N/A   

                                                plot  \
0  Suzan, an Arab- Israeli woman, must reshape he...   
1  The tale of Guendalina was pass

In [27]:
movie_df.tail()

,imdb_id,title,year,genre,director,actors,plot,country,awards,poster,rating,metascore,votes,type
38932,tt6214958,Soul Exodus,2016,Documentary,Csaba Bereczki,"Michael Alpert, Daniel Kahn, Psoy Korolenko","In search of their identities, five klezmer mu...",Hungary,1 win & 1 nomination,https://m.media-amazon.com/images/M/MV5BZmZiYW...,7.5,7.5,N/A,movie
38933,tt12268038,America's Last Little Italy: The Hill,2020,Documentary,Joseph Puleo,"Anne Agusti, Josephine Aiazzi, Keith Ballentine","The Hill in St. Louis, Missouri is an Italian ...",United States,4 wins & 2 nominations,https://m.media-amazon.com/images/M/MV5BNzM1Mz...,8.8,8.8,N/A,movie
38934,tt5946852,Janatha Garage,2016,"Action, Crime, Drama",Koratala Siva,"N.T. Rama Rao Jr., Mohanlal, Samantha Ruth Prabhu",An environmental activist comes to Hyderabad t...,India,19 wins & 7 nominations,https://m.media-amazon.com/images/M/MV5BNGUyN2...,7.2,7.2,N/A,movie
38935,tt21980046,Mrs. Buica,2023,Documentary,Eugene Buica,N/A,It's no surprise that no one ever celebrated M...,"Romania, United States",2 nominations,https://m.media-amazon.com/images/M/MV5BMjA4N2...,8.8,8.8,N/A,movie
38936,tt17886916,El Humor en Tiempos De Covid,2022,Documentary,N/A,Carlos Sanchez,About the effects of the COVID-19 pandemic on ...,Dominican Republic,N/A,N/A,N/A,N/A,N/A,movie


In [29]:
movie_df.head()

,imdb_id,title,year,genre,director,actors,plot,country,awards,poster,rating,metascore,votes,type
0,tt4076360,Third Person,2015,"Documentary, Drama",Sharon Luzon,N/A,"Suzan, an Arab- Israeli woman, must reshape he...",Israel,N/A,https://m.media-amazon.com/images/M/MV5BOTI5OT...,N/A,N/A,N/A,movie
1,tt3608918,Azzurrina,2023,Horror,Giacomo Franciosa,"Tatiana Luter, Paolo Stella, Gianfranco Terrin",The tale of Guendalina was passed down verball...,Italy,N/A,https://m.media-amazon.com/images/M/MV5BOWQ5MG...,N/A,N/A,N/A,movie
2,tt15908496,Dear Jackie,2021,Documentary,Henri Pardo,"Ronald Jones, Ivan Livingstone, Majiza Philips",Dear Jackie paints a picture of the Black comm...,Canada,N/A,https://m.media-amazon.com/images/M/MV5BNDIwYz...,N/A,N/A,N/A,movie
3,tt28378602,Zena s gumenim rukavicama,2023,Drama,Mario Sulina,"Areta Curkovic, Sandra Loncaric, Miro Cabraja",The film was inspired by the great strike of e...,Croatia,N/A,https://m.media-amazon.com/images/M/MV5BODE4NG...,N/A,N/A,N/A,movie
4,tt6340460,Los caminos de Cuba,2019,Documentary,Luciano Nacci,N/A,N/A,"Argentina, Cuba, Colombia",N/A,https://m.media-amazon.com/images/M/MV5BMzhjOT...,N/A,N/A,N/A,movie
